<a href="https://colab.research.google.com/github/mwauquier/LYSL005_machine_creativity/blob/main/LYSL005_2022_markov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Générer du texte avec des chaînes de Markov

*NB : un notebook Google Colab crée une instance de machine virtuelle à chaque utilisation. Vous devez donc relancer les installations et chargements nécessaires au lancement du code à chaque ouverture du notebook.*

Nous utiliserons pour cela le module `markovify` qui permet d'entraîner un modèle de chaînes de Markov sous Python.

Vous retrouverez la documentation au lien suivant : https://github.com/jsvine/markovify

Notez que l'auteur donne le détail annoté du code derrière le module au lien https://github.com/jsvine/markovify/blob/master/markovify/chain.py


Dans son utilisation la plus basique, le module permet la génération de texte en trois étapes :
- le chargement d'un corpus d'entraînement
- l'entraînement du modèle
- la production de texte nouveau

Pour commencer, on installe et on charge le module `markovify`

In [ ]:
!pip install markovify 

import markovify

# Notez que dans le notebook Colab, la commande "pip" est précédée d'un point d'exclamation
# Ce n'est pas le cas si vous travaillez directement depuis un terminal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 13.3 MB/s 
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18629 sha256=adcc9c060499bae30e7722b8949941c1cc3249fd31d65e9c104fe7cd0eb69ebd
  Stored in directory: /root/.cache/pip/wheels/36/c5/82/11125c5a7dadec27ef49ac2b3a12d3b1f79ff7333c92a9b67b
Successfully built markovify


On charge ensuite le corpus à partir duquel on va entraîner notre modèle (à partir duquel notre chaîne de Markov va définir ses règles probabilistes)

In [ ]:
# Code valable si vous travaillez localement sur votre machine

with open("/path/to/your/corpus.txt") as f:
    text = f.read()

Si vous souhaitez travailler directement depuis le notebook, il vous faudra téléverser le corpus sur votre Drive. Vous pourrez alors accéder à votre corpus à l'aide du code suivant. 

Attention, une autorisation d'accès à votre compte Google sera nécessaire !

In [ ]:
# Code valable si vous travaillez en ligne sur le notebook Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/path/to/your/corpus.txt') as f:
    text = f.read()

On construit le modèle

In [ ]:
text_model = markovify.Text(text)

On génère du texte

In [ ]:
print(text_model.make_sentence()) 

# En l'occurrence, ici, on l'imprime directement mais on pourra l'enregistrer dans une variable

## Quelques informations et éléments complémentaires

*Ci-dessous quelques pistes de réflexion / d'exploration pour générer du texte avec `markovify`*



### Argument

La fonction markovify.Text peut prendre l'argument 'state_size' : à quoi cela correspond-t-il ?

In [ ]:
text_model = markovify.Text(text, state_size=3)

### Longueur de l'output

Vous pouvez limiter la taille des phrases à générer avec la fonction 'make_short_sentence()' (qui prend l'argument 'max_char')


In [ ]:
print(text_model.make_short_sentence(max_chars=100))

### Combinaison de modèles

Vous pouvez créer un modèle qui combine plusieurs modèles entraînés indépendamment

In [ ]:
combined_model = markovify.combine([model_1, model_2], [1,1]) # Les nombres entre crochets correspondent au poids que vous donnez respectivement à chaque modèle

# Il s'utilise alors comme n'importe quel autre modèle
print(combined_model.make_sentence())

### Format du corpus d'entrée

Par défaut, l'algorithme exclut de l'entraînement les phrases qui contiennent des caractères jugés problématiques (parenthèses, crochets, guillements...). Plusieurs solutions sont possibles pour conserver ces phrases : les arguments 'well_formed' et 'reject_reg'.

In [ ]:
# L'argument "well_formed" est par défaut fixé à True. Le passer à False permet d'inclure par défaut les phrases 'fautives'
text_model = markovify.Text(text, well_formed=False)

# L'argument "reject_reg" permet de spécifier dans le cadre d'une expression régulière les éléments à ne pas exclure. L'utilisation de cet argument nécessite que 'well_formed' soit fixé à True

text_model = markovify.Text(text, reject_reg=r'#') # En l'occurrence, ici, on autorise les phrases contenant un #

Vous pouvez aussi pré-traiter votre corpus en amont de l'entraînement du modèle (que ce soit avec des fonctions simples à base d'expressions régulières, ou du traitement plus poussé de post-tagging, lemmatisation, etc).

### Originalité du texte généré

Par défaut, la fonction 'markovify.Text()' essaye de limiter le recouvrement entre les phrases produites et les phrases vues en entraînement. Quel est le seuil fixé par défaut ? Trouvez la réponse dans la documentation.

Ces seuils peuvent être modifiés dans la fonction 'make_sentence'.

In [ ]:
print(text_model.make_sentence(max_overlap_ratio = X)) # où X est compris entre 0 et 1

print(text_model.make_sentence(max_overlap_total = X)) # où X est un nombre entier

print(text_model.make_sentence(test_output = False)) # pour supprimer dans son entiereté l'étape de vérification

Si l'on ne veut pas modifier ce seuil, on peut jouer sur la fonction 'make_sentence' et sur le nombre d'itérations qu'elle produit afin d'éviter un recouvrement trop important, et ce à l'aide de l'argument "tries". La valeur par défaut est 10.

In [ ]:
print(text_model.make_sentence(tries=100))

### Traitement des gros corpus

Par défaut, le module charge et garde en mémoire votre corpus, notamment afin de comparer les phrases vues et celles générées. Cela peut être problématique si vous travaillez avec un corpus de grande taille.

Pour résoudre cela, vous pouvez signifier au module de ne pas garder en mémoire le corpus. il suffit alors de préciser l'argument 'retain_original' et de le fixer à False lors de l'entraînement

In [ ]:
with open("path/to/my/huge/corpus.txt") as f:
    text_model = markovify.Text(f, retain_original=False)

print(text_model.make_sentence())

Une seconde façon de faire consiste à lire le corpus ligne par ligne, à entraîner pour chaque ligne un modèle, et à combiner ces modèles pour n'en faire qu'un seul. Regardez la documentation pour une idée de l'implémentation.

### Génération du modèle

Le code comprend la fonction `markovify.NewlineText()` qui s'utilise en lieu et place de la fonction `markovify.Text` : quelle est la différence ?

In [ ]:
newline_model = markovify.NewlineText

### Divers

Notez que vous pouvez exporter le modèle complet, ou simplement la chaîne de Markov sous-jacente, afin de pouvoir les réutiliser ultérieurement. Reportez-vous à la documentation pour plus de détails.

Si vous souhaitez travailler avec des corpus (en anglais) directement interrogeables en ligne, vous pouvez utiliser le module 'nltk', et plus précisément le package 'gutenberg' (qui donne accès à des oeuvres comme des pièces de Shakespeare, des romans de Austen, Carroll, etc). De nombreuses ressources sont disponibles en ligne à ce sujet.